<a href="https://colab.research.google.com/github/Austin-rgb/deep-learning/blob/main/assignment9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import numpy as np


In [2]:
# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.expand_dims(x_train, axis=-1)  # Add channel dimension
x_test = np.expand_dims(x_test, axis=-1)
x_train = tf.image.grayscale_to_rgb(tf.convert_to_tensor(x_train))
x_test = tf.image.grayscale_to_rgb(tf.convert_to_tensor(x_test))
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)


11490434/11490434 [==============================] - 2s 0us/step


In [3]:
# Create an input layer matching MNIST dataset shape
input_tensor = Input(shape=(28, 28, 3))

In [4]:

# Load pre-trained VGG16 model without top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))



58889256/58889256 [==============================] - 4s 0us/step


In [5]:

# Freeze the weights of the convolutional base of the VGG16 model
for layer in base_model.layers:
    layer.trainable = False


In [6]:
# Add new layers on top of the pre-trained base
input_tensor = Input(shape=(224, 224, 3))
vgg_output = base_model(input_tensor)
flatten_layer = Flatten()(vgg_output)
dense_layer = Dense(256, activation='relu')(flatten_layer)
output_layer = Dense(10, activation='softmax')(dense_layer)

In [7]:
# Concatenate the pre-trained model with new model
model = Model(inputs=input_tensor, outputs=output_layer)


In [8]:

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 21140042 (80.64 MB)
Trainable params: 6425354 (24.51 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [9]:
# Preprocess and resize images in batches (Due to memory problem that comes with processing rgb dataset as a whole)
batch_size = 256
num_train_samples = len(x_train)
num_test_samples = len(x_test)
for i in range(0, num_train_samples, batch_size):
    batch_x_train = tf.image.resize(x_train[i:i+batch_size], (224, 224))
    batch_y_train = y_train[i:i+batch_size]
    model.fit(batch_x_train, batch_y_train, epochs=1, batch_size=batch_size, validation_split=0.1, verbose=2)


1/1 - 47s - loss: 11.3432 - accuracy: 0.1435 - val_loss: 26.5050 - val_accuracy: 0.5000 - 47s/epoch - 47s/step
1/1 - 1s - loss: 30.0496 - accuracy: 0.4478 - val_loss: 24.5273 - val_accuracy: 0.4231 - 1s/epoch - 1s/step
1/1 - 1s - loss: 19.1603 - accuracy: 0.5435 - val_loss: 5.6861 - val_accuracy: 0.7308 - 1s/epoch - 1s/step
1/1 - 1s - loss: 7.1834 - accuracy: 0.7130 - val_loss: 5.0677 - val_accuracy: 0.7308 - 1s/epoch - 1s/step
1/1 - 1s - loss: 5.3267 - accuracy: 0.7826 - val_loss: 1.0874 - val_accuracy: 0.9615 - 1s/epoch - 1s/step
1/1 - 1s - loss: 1.8776 - accuracy: 0.8739 - val_loss: 3.5626 - val_accuracy: 0.8846 - 1s/epoch - 1s/step
1/1 - 1s - loss: 1.5051 - accuracy: 0.9261 - val_loss: 4.2394 - val_accuracy: 0.8077 - 1s/epoch - 1s/step
1/1 - 1s - loss: 1.1065 - accuracy: 0.9261 - val_loss: 1.5617 - val_accuracy: 0.9231 - 1s/epoch - 1s/step
1/1 - 1s - loss: 1.4727 - accuracy: 0.9174 - val_loss: 0.0037 - val_accuracy: 1.0000 - 1s/epoch - 1s/step
1/1 - 1s - loss: 1.8127 - accuracy: 0.

In [11]:
# Resize test images to match input layer
x_test = tf.image.resize(x_test,(224,224))
# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


313/313 [==============================] - 48s 133ms/step - loss: 0.1465 - accuracy: 0.9853
Test Loss: 0.14653505384922028, Test Accuracy: 0.9853000044822693
